In [1]:
from scavenger import *
import pandas as pd
from geopy.distance import geodesic

In [2]:
locations = (Locations
         .select()
         .dicts())

locations = pd.DataFrame(locations)
locations.head()

,dis_lat,dis_long,index,map_avg_lat,map_avg_long,map_bot_lat,map_bot_long,map_top_lat,map_top_long,nav_lat,nav_long,radius,street
0,51.69252,5.29056,613,51.692300,5.290700,51.69208,5.29106,51.69252,5.29034,51.69252,5.29056,None,'s-Gravesandestraat
1,51.69811,5.32645,461,51.698375,5.326125,51.69807,5.32646,51.69868,5.32579,51.69811,5.32645,None,53rd Welsh Divisionplein
2,51.69938,5.30835,74,51.699270,5.311895,51.69886,5.31958,51.69968,5.30421,51.69938,5.30835,None,Aartshertogenlaan
3,51.69049,5.31681,323,51.690490,5.316805,51.69025,5.31715,51.69073,5.31646,51.69049,5.31681,None,Aastraat
4,51.69124,5.31632,178,51.691240,5.316315,51.69095,5.31676,51.69153,5.31587,51.69124,5.31632,None,Aawal


In [3]:
def add_radius_to_df(dataframe):
    '''
        Calculates the distance in meters between the map_top coordinates and the map_avg coordinates. 
    '''
    radius = []

    for idx, row in dataframe.iterrows():
        map_top = (row['map_top_lat'], row['map_top_long'])
        map_avg = (row['map_avg_lat'], row['map_avg_long'])
        radius.append(geodesic(map_top, map_avg).meters)
    
    return dataframe.assign(radius = radius)

In [4]:
locations = add_radius_to_df(locations)
locations.head()

,dis_lat,dis_long,index,map_avg_lat,map_avg_long,map_bot_lat,map_bot_long,map_top_lat,map_top_long,nav_lat,nav_long,radius,street
0,51.69252,5.29056,613,51.692300,5.290700,51.69208,5.29106,51.69252,5.29034,51.69252,5.29056,34.914066,'s-Gravesandestraat
1,51.69811,5.32645,461,51.698375,5.326125,51.69807,5.32646,51.69868,5.32579,51.69811,5.32645,41.087457,53rd Welsh Divisionplein
2,51.69938,5.30835,74,51.699270,5.311895,51.69886,5.31958,51.69968,5.30421,51.69938,5.30835,533.272174,Aartshertogenlaan
3,51.69049,5.31681,323,51.690490,5.316805,51.69025,5.31715,51.69073,5.31646,51.69049,5.31681,35.807668,Aastraat
4,51.69124,5.31632,178,51.691240,5.316315,51.69095,5.31676,51.69153,5.31587,51.69124,5.31632,44.586598,Aawal


In [5]:
# Writes radius column from locations dataframe to locations column in database
for idx, row in locations.iterrows():
    query = Locations.update(radius = row['radius']).where(Locations.index == row['index'])
    query.execute()